### Analysis of ...

In [1]:
import pandas as pd
import glob
from tqdm import tqdm


In [2]:
memory_df = pd.read_csv('../data/01_interview_data/recall_vr_driving_e.csv')
memory_df.head()

,subject,binge_drinking,buckle_up,diabetes,HIV,text_driving,drugged_driving,smoking,sun_protection,wash_hands,...,marijuana,tech_addiction,healthy_diet,brunch,burger,coffee,education_donation,furniture,hotel,condition
0,sub001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,sub002,1,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,sub003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,sub004,0,0,0,0,0,1,0,0,0,...,1,0,1,1,0,1,0,0,1,1
4,sub005,0,0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1


In [3]:

subj_restrict_counter = 39
memory_df = memory_df.loc[:subj_restrict_counter]
print(len(memory_df))
memory_df.head()

40


,subject,binge_drinking,buckle_up,diabetes,HIV,text_driving,drugged_driving,smoking,sun_protection,wash_hands,...,marijuana,tech_addiction,healthy_diet,brunch,burger,coffee,education_donation,furniture,hotel,condition
0,sub001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,sub002,1,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,sub003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,sub004,0,0,0,0,0,1,0,0,0,...,1,0,1,1,0,1,0,0,1,1
4,sub005,0,0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,1


In [4]:
recognition_df = pd.read_csv('../data/02_qualtrics_data/recognition_survey_vr_driving_e_cleaned.csv')#
recognition_df.tail()

,subject,age,gender,virtual_sickness_1_discomfort,virtual_sickness_2_fatigue,virtual_sickness_3_eyestrain,virtual_sickness_4_difficultyfocusing,virtual_sickness_5_headache,virtual_sickness_6_fullnessofhead,virtual_sickness_7_blurredvision,...,emo_health_5_text_drivin_1emo,emo_health_6_drugged_dri_1emo,emo_health_7_smoking_1emo,emo_health_8_sun_protect_1emo,emo_health_9_wash_hands_2emo,emo_health_10_vaccinatio_2emo,emo_health_11_vaping_2emo,emo_health_12_marijuana_2emo,emo_health_13_tech_addic_2emo,emo_health_14_healthy_di_1emo
35,sub036,18.0,2,1,1,1,1,1,1,1,...,1,1,1,1,2,2,2,2,2,1
36,sub037,19.0,1,1,1,1,2,1,1,1,...,2,2,1,1,2,2,2,2,2,1
37,sub038,19.0,2,1,1,2,3,2,1,1,...,1,1,1,1,2,2,2,2,2,1
38,sub039,21.0,2,1,1,1,1,1,1,1,...,1,1,1,1,2,2,2,2,2,1
39,sub040,22.0,1,1,1,2,1,1,2,2,...,1,1,1,1,2,2,2,2,2,1


In [5]:

recognition_df = recognition_df.drop(['age', 'gender', 'virtual_sickness_1_discomfort',
       'virtual_sickness_2_fatigue', 'virtual_sickness_3_eyestrain',
       'virtual_sickness_4_difficultyfocusing', 'virtual_sickness_5_headache',
       'virtual_sickness_6_fullnessofhead', 'virtual_sickness_7_blurredvision',
       'virtual_sickness_8_dizzy', 'virtual_sickness_9_vertigo',
       'spatial_presence_1_transported', 'spatial_presence_2_threedimensional',
       'spatial_presence_3_realisticproportions',
       'spatial_presence_4_movefreely', 'spatial_presence_5_physicallypresent', 
       'distractor_crushed ', 'distractor_crushed_certainty', 
       'distractor_climate_change', 'distractor_climate_change_certainty',
       'distractor_greek_salad', 'distractor_greek_salad_certainty',                             
       'distractor_smart_phone','distractor_smart_phone_certainty', 
       'emo_commercial_1_brunch_1emo', 'emo_commercial_2_burger_2emo', 
       'emo_commercial_3_coffee_1emo', 'emo_commercial_4_educati_1emo', 
       'emo_commercial_5_furnitu_2emo', 'emo_commercial_6_hotel_1emo', 
       'emo_health_1_binge_drink_2emo', 'emo_health_2_buckle_up_1emo', 
       'emo_health_3_diabetes_2emo', 'emo_health_4_HIV_2emo', 
       'emo_health_5_text_drivin_1emo', 'emo_health_6_drugged_dri_1emo', 
       'emo_health_7_smoking_1emo', 'emo_health_8_sun_protect_1emo', 
       'emo_health_9_wash_hands_2emo', 'emo_health_10_vaccinatio_2emo', 
       'emo_health_11_vaping_2emo', 'emo_health_12_marijuana_2emo', 
       'emo_health_13_tech_addic_2emo','emo_health_14_healthy_di_1emo'], axis=1)


recognition_df.head()

,subject,binge_drinking,binge_drinking_certainty,buckle_up,buckle_up_certainty,diabetes,diabetes_certainty,HIV,HIV_certainty,text_driving,...,burger_certainty,coffee,coffee_certainty,education_donation,education_donation_certainty,furniture,furniture_certainty,hotel,hotel_certainty,sum_false_alerts
0,sub001,2,NaN,2,NaN,2,NaN,2,NaN,2,...,NaN,2,NaN,2,NaN,2,NaN,2,NaN,1
1,sub002,1,6.0,1,2.0,1,7.0,2,NaN,1,...,1.0,1,4.0,1,1.0,2,NaN,2,NaN,0
2,sub003,2,NaN,1,4.0,2,NaN,2,NaN,2,...,6.0,2,NaN,2,NaN,2,NaN,2,NaN,0
3,sub004,1,2.0,1,3.0,1,3.0,1,6.0,1,...,4.0,1,7.0,2,NaN,2,NaN,1,2.0,0
4,sub005,1,1.0,1,7.0,1,7.0,2,NaN,1,...,7.0,1,1.0,1,4.0,2,NaN,1,7.0,1


In [6]:
folders = glob.glob("../data/00_gaze_data/sub*")
folders.sort()
subjs = []
for f in folders:
    subjs.append(f[-6:])
print(len(subjs))  

# this needs to be removed -- it is only here to restrict ppts to the ones we have in the preliminary analysis to check if everything workds
subjs = subjs[:subj_restrict_counter+1]
print(len(subjs))  
subjs[:3]

40
40


['sub001', 'sub002', 'sub003']

In [8]:
for i in tqdm(range(len(subjs))):
    #read this participant's fixation data
    curr_sub_fixation_file = '../data/00_gaze_data/fixation_billboards/'  + subjs[i] +'_fixations_billboards.csv'
    out_file_name = '../data/03_fixation_billboards_recall_recognition/'  + subjs[i] +'_fixations_billboards_recall_recognition_emointegrated.csv'
   
    curr_sub_fix_df = pd.read_csv(curr_sub_fixation_file)
    curr_sub_fix_df['billboard_image_at_position_text_emotion'] = curr_sub_fix_df.loc[:, 'billboard_image_at_position_text']
    curr_sub_fix_df = curr_sub_fix_df.replace(to_replace=r'health_\d*_', value='', regex=True)
    curr_sub_fix_df = curr_sub_fix_df.replace(to_replace=r'commercial_\d*_', value='', regex=True)
    curr_sub_fix_df['billboard_image_at_position_text'] = curr_sub_fix_df['billboard_image_at_position_text'].replace(to_replace=r'_emo', value='', regex=True)
    curr_sub_fix_df['billboard_image_at_position_text'] = curr_sub_fix_df['billboard_image_at_position_text'].replace(to_replace=r'_neu', value='', regex=True)
    curr_sub_fix_df.head()


    #subselect the position, billboard, and fixations for this subject
    position_billboard_fix = curr_sub_fix_df[["position_index", "billboard_image_at_position_text", "billboard_image_at_position_text_emotion", "fix_bill", "count"]]
    #print(position_billboard_fix['fix_bill'].values)
    #position_billboard_fix 

    #munge the recall df around
    current_subject_recall = memory_df[memory_df['subject']==subjs[i]]
    current_subject_recall = current_subject_recall.drop('subject', axis= 1)
    current_subject_recall = current_subject_recall.T
    current_subject_recall['billboard'] = current_subject_recall.index
    current_subject_recall.rename( columns={i :'recall'}, inplace=True )
    #current_subject_recall.head()


    #merge fixation and recall df
    position_billboard_fix_recall = position_billboard_fix.merge(current_subject_recall, left_on = 'billboard_image_at_position_text', right_on = 'billboard')
    position_billboard_fix_recall = position_billboard_fix_recall.drop('billboard', axis=1)
    #print(position_billboard_fix_recall['fix_bill'].values)

    #munge the recognition df around
    current_subject_recognition = recognition_df[recognition_df['subject']==subjs[i]]
    current_subject_recognition = current_subject_recognition.drop(['subject',
                                                                    'binge_drinking_certainty','buckle_up_certainty',
                                                                    'diabetes_certainty', 'HIV_certainty', 
                                                                    'text_driving_certainty', 
                                                                    'drugged_driving_certainty', 'smoking_certainty', 
                                                                    'sun_protection_certainty', 'wash_hands_certainty', 
                                                                    'vaccination_certainty', 'vaping_certainty', 
                                                                    'marijuana_certainty', 'tech_addiction_certainty', 
                                                                    'healthy_diet_certainty', 'brunch_certainty', 
                                                                    'burger_certainty', 'coffee_certainty', 
                                                                    'education_donation_certainty', 'furniture_certainty', 
                                                                    'hotel_certainty'], axis= 1)
    current_subject_recognition = (current_subject_recognition.T - 2) * (-1)  #original survey was coded as 1-recognize 2-no-reconition, so that makes it into 1=yes, 0=no
    current_subject_recognition['billboard'] = current_subject_recognition.index
    current_subject_recognition.rename( columns={i :'recognition'}, inplace=True )
    current_subject_recognition.head()

    #merge the fixation-recall and recognition df
    position_billboard_fix_recall_recognition = position_billboard_fix_recall.merge(current_subject_recognition, left_on = 'billboard_image_at_position_text', right_on = 'billboard')
    position_billboard_fix_recall_recognition = position_billboard_fix_recall_recognition.drop('billboard', axis=1)
    position_billboard_fix_recall_recognition['recall'] = position_billboard_fix_recall_recognition['recall'].astype(int)
    #position_billboard_fix_recall_recognition['sum_fa'] = recognition_df[recognition_df['subject']==subjs[i]]['sum_fa'].values[0]


    #save to file
    position_billboard_fix_recall_recognition.to_csv(out_file_name)
    #position_billboard_fix_recall.to_csv(out_file_name)
    #print('---')
    
    
#print('done')

100%|████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 234.52it/s]


In [9]:
position_billboard_fix_recall_recognition

,position_index,billboard_image_at_position_text,billboard_image_at_position_text_emotion,fix_bill,count,recall,recognition
0,1,burger,burger_neu,1.0,1.0,0,0
1,2,tech_addiction,tech_addiction_emo,2.0,1.0,0,1
2,3,text_driving,text_driving_neu,3.0,1.0,0,1
3,4,education_donation,education_donation_emo,4.0,1.0,0,0
4,5,vaping,vaping_emo,5.0,2.0,1,1
5,6,drugged_driving,drugged_driving_emo,0.0,0.0,0,1
6,7,diabetes,diabetes_emo,0.0,0.0,0,0
7,8,marijuana,marijuana_neu,8.0,2.0,1,1
8,9,wash_hands,wash_hands_neu,0.0,0.0,0,1
9,10,binge_drinking,binge_drinking_neu,0.0,0.0,1,1


In [10]:
out_file_name

'../data/03_fixation_billboards_recall_recognition/sub040_fixations_billboards_recall_recognition_emointegrated.csv'